### Question 1. Understanding docker first run

Run docker with the python:3.12.8 image in an interactive mode, use the entrypoint bash.

What's the version of pip in the image?

```
24.3.1
24.2.1
23.3.1
23.2.1
```

#### Answer:  24.3.1

### Question 2. Understanding Docker networking and docker-compose

Given the following docker-compose.yaml, what is the hostname and port that pgadmin should use to connect to the postgres database?

```
services:
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment:
      POSTGRES_USER: 'postgres'
      POSTGRES_PASSWORD: 'postgres'
      POSTGRES_DB: 'ny_taxi'
    ports:
      - '5433:5432'
    volumes:
      - vol-pgdata:/var/lib/postgresql/data

  pgadmin:
    container_name: pgadmin
    image: dpage/pgadmin4:latest
    environment:
      PGADMIN_DEFAULT_EMAIL: "pgadmin@pgadmin.com"
      PGADMIN_DEFAULT_PASSWORD: "pgadmin"
    ports:
      - "8080:80"
    volumes:
      - vol-pgadmin_data:/var/lib/pgadmin  

volumes:
  vol-pgdata:
    name: vol-pgdata
  vol-pgadmin_data:
    name: vol-pgadmin_data
```

```
- postgres:5433
- localhost:5432
- db:5433
- postgres:5432
- db:5432
```

If there are more than one answers, select only one of them

#### Answer: db:5432

### Question 3. Trip Segmentation Count

During the period of October 1st 2019 (inclusive) and November 1st 2019 (exclusive), how many trips, respectively, happened:

Up to 1 mile
In between 1 (exclusive) and 3 miles (inclusive),
In between 3 (exclusive) and 7 miles (inclusive),
In between 7 (exclusive) and 10 miles (inclusive),
Over 10 miles
Answers:

104,802; 197,670; 110,612; 27,831; 35,281
104,802; 198,924; 109,603; 27,678; 35,189
104,793; 201,407; 110,612; 27,831; 35,281
104,793; 202,661; 109,603; 27,678; 35,189
104,838; 199,013; 109,645; 27,688; 35,202

The SQL query is following:

```
SELECT
    SUM(CASE WHEN trip_distance <= 1 THEN 1 ELSE 0 END) AS trips_up_to_1_mile,
    SUM(CASE WHEN trip_distance > 1 AND trip_distance <= 3 THEN 1 ELSE 0 END) AS trips_1_to_3_miles,
    SUM(CASE WHEN trip_distance > 3 AND trip_distance <= 7 THEN 1 ELSE 0 END) AS trips_3_to_7_miles,
    SUM(CASE WHEN trip_distance > 7 AND trip_distance <= 10 THEN 1 ELSE 0 END) AS trips_7_to_10_miles,
    SUM(CASE WHEN trip_distance > 10 THEN 1 ELSE 0 END) AS trips_over_10_miles
FROM
    yellow_taxi_trips
WHERE
    CAST(tpep_pickup_datetime AS DATE) >= '2019-10-01'
    AND CAST(tpep_pickup_datetime AS DATE) < '2019-11-01';

```

#### Answer: 104,838; 199,013; 109,645; 27,688; 35,202

### Question 4. Longest trip for each day

Which was the pick up day with the longest trip distance? Use the pick up time for your calculations.

Tip: For every day, we only care about one single trip with the longest distance.

2019-10-11
2019-10-24
2019-10-26
2019-10-31

The SQL query is following:

```
SELECT 
    DATE(tpep_dropoff_datetime) AS pickup_day,
    MAX(trip_distance) AS longest_trip_distance
FROM 
    yellow_taxi_trips
GROUP BY 
    DATE(tpep_dropoff_datetime)
ORDER BY 
    longest_trip_distance DESC

```

#### Answer: 2019-10-11

### Question 5. Three biggest pickup zones

Which were the top pickup locations with over 13,000 in total_amount (across all trips) for 2019-10-18?

Consider only lpep_pickup_datetime when filtering by date.

East Harlem North, East Harlem South, Morningside Heights
East Harlem North, Morningside Heights
Morningside Heights, Astoria Park, East Harlem South
Bedford, East Harlem North, Astoria Park

The SQL query is following:

```
SELECT 
    z."Zone" AS pickup_zone,
    SUM(g.total_amount) AS total_amount
FROM 
    green_taxi_data g
JOIN 
    zones z
ON 
    g."PULocationID" = z."LocationID"
WHERE 
    DATE(g.lpep_pickup_datetime) = '2019-10-18'
GROUP BY 
    z."Zone"
HAVING 
    SUM(g.total_amount) > 13000
ORDER BY 
    total_amount DESC;

```

#### Answer: East Harlem North, East Harlem South, Morningside Heights

### Question 6. Largest tip

For the passengers picked up in October 2019 in the zone named "East Harlem North" which was the drop off zone that had the largest tip?

Note: it's tip , not trip

We need the name of the zone, not the ID.

Yorkville West
JFK Airport
East Harlem North
East Harlem South

The SQL query is following:

```
SELECT 
    z_drop."Zone" AS dropoff_zone,
    MAX(g.tip_amount) AS largest_tip
FROM 
    green_taxi_data g
JOIN 
    zones z_pick ON g."PULocationID" = z_pick."LocationID"
JOIN 
    zones z_drop ON g."DOLocationID" = z_drop."LocationID"
WHERE 
    z_pick."Zone" = 'East Harlem North'
    AND g.lpep_pickup_datetime >= '2019-10-01 00:00:00'
    AND g.lpep_pickup_datetime < '2019-11-01 00:00:00'
GROUP BY 
    z_drop."Zone"
ORDER BY 
    largest_tip DESC
LIMIT 1;

```

#### Answer: JFK Airport

### Question 7. Terraform Workflow

Which of the following sequences, respectively, describes the workflow for:

Downloading the provider plugins and setting up backend,
Generating proposed changes and auto-executing the plan
Remove all resources managed by terraform`
Answers:

terraform import, terraform apply -y, terraform destroy
teraform init, terraform plan -auto-apply, terraform rm
terraform init, terraform run -auto-approve, terraform destroy
terraform init, terraform apply -auto-approve, terraform destroy
terraform import, terraform apply -y, terraform rm

#### Answer: terraform init, terraform apply -auto-approve, terraform destroy